# Processor of Camera Data

In [21]:
# imports
import numpy as np
import pathlib, importlib, logging, datetime, json, platform
from threading import Thread
from openmsitoolbox.logging import OpenMSILogger
from openmsistream import (
    DataFileDownloadDirectory,
    DataFileStreamProcessor,
    MetadataJSONReproducer,
)

In [6]:
# Configure a logger (only needed when running in a Jupyter notebook like this)
logger = OpenMSILogger("OpenMSIConsumers", filelevel=None)
importlib.reload(logging)

<module 'logging' from '/Users/namanparikh/opt/anaconda3/envs/openmsi/lib/python3.9/logging/__init__.py'>

In [27]:
# The name of the topic to consume files from
CONSUMER_TOPIC_NAME = "tutorial_data"
TOPIC_NAME = "tutorial_metadata"

# Path to the root directory of this repo
repo_root_dir = pathlib.Path().resolve().parent

In [28]:
# Paths to the config file and the directory holding the test files
CONFIG_FILE_PATH = repo_root_dir / "streaming_2" / "config_files" / "confluent_cloud_broker.config"
TEST_RECO_DIR = repo_root_dir.parent / "streaming_2" / "reconstructed_test_files"

## Stream Processor

In [35]:
class PlaceholderStreamProcessor(DataFileStreamProcessor):
    """Performs a placeholder task (writing out a file to the local system) for every
    data file reconstructed from a topic
    """

    def _process_downloaded_data_file(self, datafile, lock):
        "Writes out a file with a timestamp for each reconstructed file"
        try:
            timestamp = datetime.datetime.now()
            rel_filepath = datafile.relative_filepath
            rel_fp_str = str(rel_filepath.as_posix()).replace("/","_").replace(".","_")
            output_filepath = self._output_dir / f"{rel_fp_str}_placeholder.npy"
            with lock:
                arr = np.zeros((3,3))
                np.save(output_filepath, arr)
                upload_file = UploadDataFile(output_filepath, rootdir=self._output_dir, logger=logger)
                upload_file.upload_whole_file(CONFIG_FILE_PATH, TOPIC_NAME)
        except Exception as exc:
            return exc
        return None
    
    @classmethod
    def run_from_command_line(cls, args=None):
        "Not used in this example... stay tuned for the live coding tomorrow!"
        pass

In [36]:
def stream_processor_task(stream_processor):
    """Run "process_files_as_read" for the given stream processor, and log a message
    when it gets shuts down
    
    Args:
        stream_processor (openmsistream.DataFileStreamProcessor): The stream processor to run
    """
    # This call to "process_files_as_read" hangs until the stream processor is shut down
    (
        n_m_r, # The number of messages read
        n_m_p, # The number of messages processed
        n_f_p, # The number of files successfully processed
        p_fps, # Paths to the most recently-processed files
    ) = stream_processor.process_files_as_read()
    stream_processor.close()
    msg = f"{n_m_r} total messages were consumed"
    if n_f_p > 0:
        msg += (
            f", {n_m_p} messages were processed,"
            f" and {n_f_p} files were successfully processed"
        )
    else:
        msg += f" and {n_m_p} messages were successfully processed"
    msg += (
        f". Up to {stream_processor.N_RECENT_FILES} most recently "
        "processed files:\n\t"
    )
    msg += "\n\t".join([str(fp) for fp in p_fps])
    stream_processor.logger.info(msg)

In [37]:
# Path to the directory to store the StreamProcessor output
STREAM_PROCESSOR_OUTPUT_DIR = repo_root_dir / "streaming_2" / "processor_1"

In [32]:
STREAM_PROCESSOR_OUTPUT_DIR

PosixPath('/Users/namanparikh/Documents/GitHub/paradim/reu2024-hyperspectral-camera/streaming_2/processor_1')

In [38]:
# Create the StreamProcessor
psp = PlaceholderStreamProcessor(
    CONFIG_FILE_PATH,
    CONSUMER_TOPIC_NAME,
    output_dir=STREAM_PROCESSOR_OUTPUT_DIR,
    logger=logger,
)
# Start running its "process_files_as_read" function in a separate thread
processor_thread = Thread(
    target=stream_processor_task,
    args=(psp,),
)
processor_thread.start()

[OpenMSIConsumers 2024-07-18 15:24:23] Log files and output will be in /Users/namanparikh/Documents/GitHub/paradim/reu2024-hyperspectral-camera/streaming_2/processor_1
[OpenMSIConsumers 2024-07-18 15:24:23] Will process files from messages in the tutorial_data topic using 2 threads
[OpenMSIConsumers 2024-07-18 15:24:25] Uploading /Users/namanparikh/Documents/GitHub/paradim/reu2024-hyperspectral-camera/streaming_2/processor_1/test_data_txt_placeholder.npy to tutorial_metadata in 524288-byte chunks using 2 threads....
[OpenMSIConsumers 2024-07-18 15:24:25] Waiting for all enqueued messages to be delivered (this may take a moment)....
[OpenMSIConsumers 2024-07-18 15:24:25] Done uploading /Users/namanparikh/Documents/GitHub/paradim/reu2024-hyperspectral-camera/streaming_2/processor_1/test_data_txt_placeholder.npy
[OpenMSIConsumers 2024-07-18 15:24:25] Uploading /Users/namanparikh/Documents/GitHub/paradim/reu2024-hyperspectral-camera/streaming_2/processor_1/_ipynb_checkpoints_test_data-chec

In [34]:
# Manually shut down the stream processor (if running from the command line this would
# be like typing "q" in the Terminal window)
psp.control_command_queue.put("q")
processor_thread.join()